# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests


# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bocana de paiwas,12.7857,-85.1227,25.24,91,100,4.42,NI,1722553963
1,1,chos malal,-37.3781,-70.2709,11.43,45,100,7.23,AR,1722553964
2,2,arica,-18.4750,-70.3042,18.75,77,0,3.09,CL,1722553609
3,3,haiku-pauwela,20.9219,-156.3051,28.98,62,0,10.29,US,1722553966
4,4,gornozavodsk,58.3758,58.3211,12.43,98,99,1.04,RU,1722553968


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng', 'Lat',  
    size='Humidity',  
    geo=True,  
    tiles='OSM', 
    frame_width=800,  
    frame_height=600, 
    color='Country',  
    alpha=0.6,  
    hover_cols=['City', 'Humidity']  
)

# Display the map
map_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Country,Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
city_temp_1 = city_data_df.loc[city_data_df['Max Temp']<27,:]
city_temp_2 = city_temp_1.loc[city_temp_1['Max Temp']>21,:]
city_wind_sp = city_temp_2.loc[city_temp_2['Wind Speed']<4.5,:]
city_cloudiness = city_wind_sp.loc[city_wind_sp['Cloudiness']==0,:]
city_cloudiness

# Drop any rows with null values
city_ideal = city_cloudiness.dropna()

# Display sample data
city_ideal

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
98,98,remire-montjoly,4.9167,-52.2667,26.02,89,0,1.03,GF,1722554083
133,133,la rinconada,37.4873,-5.9826,26.18,68,0,4.12,ES,1722554127
213,213,hohhot,40.8106,111.6522,21.16,83,0,3.00,CN,1722554228
221,221,avanos,38.7150,34.8467,21.07,61,0,1.40,TR,1722554237
256,256,ajdabiya,30.7554,20.2263,25.57,62,0,4.22,LY,1722554279
294,294,as-suwayda,32.7090,36.5695,21.43,50,0,1.62,SY,1722554330
323,323,mana,5.6592,-53.7767,26.42,82,0,2.58,GF,1722554366
328,328,mizdah,31.4451,12.9801,23.76,56,0,2.60,LY,1722554373
342,342,corbelia,-24.7989,-53.3067,22.64,57,0,3.30,BR,1722554391
344,344,tarija,-21.5355,-64.7296,23.95,20,0,2.06,BO,1722554393


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_ideal.loc[:,['City','Country','Lat','Lng','Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
98,remire-montjoly,GF,4.9167,-52.2667,89,
133,la rinconada,ES,37.4873,-5.9826,68,
213,hohhot,CN,40.8106,111.6522,83,
221,avanos,TR,38.7150,34.8467,61,
256,ajdabiya,LY,30.7554,20.2263,62,
294,as-suwayda,SY,32.7090,36.5695,50,
323,mana,GF,5.6592,-53.7767,82,
328,mizdah,LY,31.4451,12.9801,56,
342,corbelia,BR,-24.7989,-53.3067,57,
344,tarija,BO,-21.5355,-64.7296,20,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey":geoapify_key,
    "categories":"accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lon = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"circle:{lon},{lat},{radius}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
remire-montjoly - nearest hotel: No hotel found
la rinconada - nearest hotel: No hotel found
hohhot - nearest hotel: No hotel found
avanos - nearest hotel: No hotel found
ajdabiya - nearest hotel: No hotel found
as-suwayda - nearest hotel: No hotel found
mana - nearest hotel: No hotel found
mizdah - nearest hotel: No hotel found
corbelia - nearest hotel: No hotel found
tarija - nearest hotel: No hotel found
tucurui - nearest hotel: No hotel found
vine grove - nearest hotel: No hotel found
dowlatabad - nearest hotel: No hotel found
granada - nearest hotel: No hotel found
gargalianoi - nearest hotel: No hotel found
mir bachah kot - nearest hotel: No hotel found
beruniy - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
98,remire-montjoly,GF,4.9167,-52.2667,89,No hotel found
133,la rinconada,ES,37.4873,-5.9826,68,No hotel found
213,hohhot,CN,40.8106,111.6522,83,No hotel found
221,avanos,TR,38.7150,34.8467,61,No hotel found
256,ajdabiya,LY,30.7554,20.2263,62,No hotel found
294,as-suwayda,SY,32.7090,36.5695,50,No hotel found
323,mana,GF,5.6592,-53.7767,82,No hotel found
328,mizdah,LY,31.4451,12.9801,56,No hotel found
342,corbelia,BR,-24.7989,-53.3067,57,No hotel found
344,tarija,BO,-21.5355,-64.7296,20,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    alpha =0.8,
    color = 'City',
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)